In [1]:
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 讀取資料
train_raw = os.path.join('./data-science-london-scikit-learn/', 'train.csv')
label_raw = os.path.join('./data-science-london-scikit-learn/', 'trainLabels.csv')
test_raw = os.path.join('./data-science-london-scikit-learn/', 'test.csv')
train = pd.read_csv(train_raw, header=None)
label = pd.read_csv(label_raw, header=None)
test = pd.read_csv(test_raw, header=None)
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train, label, test_size=0.25, random_state=42)
# 建立模型
clf = GradientBoostingClassifier(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.88


In [4]:
# 設定要訓練的超參數組合
n_estimators = range(1,1000,7)
max_depth = range(3,10,1)
param_dist = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
random_search = RandomizedSearchCV(clf, param_dist, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
search_result = random_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   22.1s finished


In [5]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=search_result.best_params_['max_depth'],
                                           n_estimators=search_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [6]:
# 調整參數後的預測結果更正確了
print(metrics.accuracy_score(y_test, y_pred))

0.892


In [7]:
# 使用測試集進行預測並輸出
result = clf_bestparam.predict(test)
result_df = pd.DataFrame({'Id':range(1,result.size+1),'Solution':result})
result_df.to_csv('./Day_048_HW_result.csv', encoding='utf-8', index=False)